In [ ]:
import geopandas as gpd
import json
import matplotlib.pyplot as plt
import pandas as pd
from scientific_python_utils.geospatial import ensure_projected_CRS

from pathlib import Path

In [ ]:
GEOSPATIAL_MAPS_FOLDER = Path("/ofo-share/scratch-david/NRS-all-sites/geospatial_maps")
SHIFT_PER_DATASET = "/ofo-share/repos-david/UCNRS-experiments/data/shift_per_dataset.json"
METADATA_FILE = "/ofo-share/drone-imagery-organization/3c_metadata-extracted/all-mission-polygons-w-metadata.gpkg"
OUTPUT_FILE = "/ofo-share/repos-david/UCNRS-experiments/data/all_geospatial_maps_merged.geojson"

# Simplify the geometry such that the maximum deviation never exceeds this amount
SIMPLIFY_TOL = 0.1
BUFFER_AMOUNT = -0.01
VIS = True

# Allows you to specify a subset of the data for testing
START_IND = 0
STOP_IND = 100

In [ ]:
missions_metadata = gpd.read_file(METADATA_FILE)

In [ ]:
from IPython.core.debugger import set_trace
# List all the files that are present
map_files = sorted(GEOSPATIAL_MAPS_FOLDER.glob("*"))[START_IND:STOP_IND]

# Open the list of shifts for each dataset
with open(SHIFT_PER_DATASET, "r") as infile:
    shifts_per_dataset = json.load(infile)

# Iterate over all datasets. Read them and post-process the results
all_dataframes = []
for map_file in map_files:
    mission_id = map_file.stem

    if mission_id not in shifts_per_dataset:
        print(f"Mission ID {mission_id} not in shifts_per_dataset. Skipping.")
        continue

    # Try to read the dataset from disk
    print(f"Reading {map_file}")
    try:
        gdf = gpd.read_file(map_file)
    except:
        print(f"Failed to read {mission_id}. Skipping")
        continue

    # Make sure this is in a projected CRS so geometric operations work as expected
    gdf = ensure_projected_CRS(gdf)
    # Simplify the geometry to make future operations faster
    print("Simplifying ")
    gdf.geometry = gdf.simplify(SIMPLIFY_TOL)
    # Make sure the geometry is valid. This is especially important because this data is generated
    # from the projection of the mesh.
    #gdf.geometry = gdf.make_valid()
    print("Buffering")
    gdf.geometry = gdf.buffer(BUFFER_AMOUNT)
    # Add a dataset identifier
    gdf["mission_id"] = mission_id

    # Extract the corresponding mission-level metadata which contains the bounding flight polygon
    metadata_for_mission = missions_metadata[missions_metadata.mission_id == mission_id]
    # Make the CRS for the flight polygon match that of the prediction
    metadata_for_mission.to_crs(gdf.crs, inplace=True)
    # Extract the actual polygon element
    mission_polygon = metadata_for_mission.geometry.values[0]

    print("before clip")
    print(gdf.geometry)
    # Clip the prediction to the extent of the mission polygon
    gdf = gpd.clip(gdf=gdf, mask=mission_polygon, keep_geom_type=True)
    # Undo the buffering ammount that was done initially
    gdf.geometry = gdf.buffer(-BUFFER_AMOUNT)
    print("After clip")
    print(gdf.geometry)

    # Get the x, y shift for this dataset
    shift = shifts_per_dataset[mission_id]
    # Apply this shift. Note it's important this happens after the crop occurs since the flight
    # polygon is relative to the input data, not absolute.
    gdf.geometry = gdf.translate(xoff=shift[0], yoff=shift[1])

    # Visualize if requested
    if VIS:
        f, ax = plt.subplots()
        metadata_for_mission.boundary.plot(ax=ax)
        gdf.plot("class_names", legend=True, vmin=-0.5, vmax=9.5, ax=ax)
        plt.show()

    all_dataframes.append(gdf)


In [ ]:
merged_dataframes = pd.concat(all_dataframes)
print(merged_dataframes)

In [ ]:
merged_dataframes.to_file(OUTPUT_FILE)
print("done")